In [8]:
import msgpack
import pyxdf
import pandas as pd
import numpy as np

### 1. .pldata files


In [2]:
def decode_data(item):
    if isinstance(item, bytes):
        try:
            # Attempt to decode the bytes as UTF-8
            return item.decode('utf-8')
        except UnicodeDecodeError:
            try:
                # Attempt to unpack nested msgpack data
                return msgpack.unpackb(item, raw=False)
            except Exception:
                # Return original bytes if both attempts fail
                return item
    elif isinstance(item, list):
        # Recursively decode items in the list
        return [decode_data(sub_item) for sub_item in item]
    elif isinstance(item, dict):
        # Recursively decode keys and values in the dictionary
        return {decode_data(key): decode_data(value) for key, value in item.items()}
    else:
        # Return the item as is if it's neither bytes, list, nor dict
        return item

def load_pldata(file_path):
    with open(file_path, 'rb') as f:
        unpacker = msgpack.Unpacker(f, raw=False)
        data = []
        for unpacked in unpacker:
            data.append(decode_data(unpacked))
    return data


In [3]:
file = '/Volumes/psp_data/PSP/data/raw/Data/sub-PSP_031/ses-S001/2021_12_03/003/gaze.pldata'
data = load_pldata(file)

### 2. read xdf files

In [5]:
path1 = '/Volumes/psp_data/PSP/data/raw/Data/sub-PSP_034/ses-S001/RAW_Streams/ses-S001/sub-PSP_034_ses-S001_task-rest_001_run-003_eeg.xdf'
streams, header = pyxdf.load_xdf(path1)

In [9]:
# Find out which stream contains pupil data and make data to pd.DataFrame

pupil_stream = streams[1]

df = pd.DataFrame(pupil_stream['time_series'])
cols = [i['label'][0] for i in pupil_stream['info']['desc'][0]['channels'][0]['channel']]
cols = np.array(cols)
df.columns = cols

time = pd.DataFrame(pupil_stream['time_stamps'])
df['time'] = time.iloc[:, 0].values
df.time = df.time - df.time.values[0]

df.loc[:, 'time'] = np.round(df.time * 1000)
df = df.set_index(pd.to_datetime(df['time'], unit='ms'))

In [11]:
df.columns

Index(['confidence', 'norm_pos_x', 'norm_pos_y', 'gaze_point_3d_x',
       'gaze_point_3d_y', 'gaze_point_3d_z', 'eye_center0_3d_x',
       'eye_center0_3d_y', 'eye_center0_3d_z', 'eye_center1_3d_x',
       'eye_center1_3d_y', 'eye_center1_3d_z', 'gaze_normal0_x',
       'gaze_normal0_y', 'gaze_normal0_z', 'gaze_normal1_x', 'gaze_normal1_y',
       'gaze_normal1_z', 'diameter0_2d', 'diameter1_2d', 'diameter0_3d',
       'diameter1_3d', 'time'],
      dtype='object')

### 3. load matlab files (e.g. by molly)


In [12]:
def load_new_psp(input_mat):
    mat = scipy.io.loadmat(file)

    left = pd.DataFrame({'time': mat['eye'][0][0][1][0][1][0],
     'diameter': mat['eye'][0][0][3][0][1][0],
     'confidence': mat['eye'][0][0][2][0][1][0],
     'x': mat['eye'][0][0][4][0][1][0],
     'y': mat['eye'][0][0][5][0][1][0]})
    gaze = left.loc[:, ['time', 'confidence', 'x', 'y']].rename(columns = {'time': 'timestamp'})
    left = left.loc[:, ['time', 'confidence', 'diameter']].rename(columns = {'time': 'timestamp'})
    left.loc[:, 'id'] = 1

    right = pd.DataFrame({'time': mat['eye'][0][0][1][0][0][0],
     'diameter': mat['eye'][0][0][3][0][0][0],
     'confidence': mat['eye'][0][0][2][0][0][0],
     'x': mat['eye'][0][0][4][0][0][0],
     'y': mat['eye'][0][0][5][0][0][0]})
    right = right.loc[:, ['time', 'confidence', 'diameter']].rename(columns = {'time': 'timestamp'})
    right.loc[:, 'id'] = 0
    return gaze, right, left